In [ ]:
#!/usr/bin/env python3 
#！！final one that I use to generate the wind input files for 5m p s for wave filed running input
# Hurricane wind + wind-stress for MOM6 data_table ("bilinear")
# - Output: Taux/Tauy/u10/v10(Time, Lat, Lon) with 1-D Lat/Lon in degrees
# - Time axis includes a +1 record tail to avoid "after range" errors

import os
import numpy as np
from netCDF4 import Dataset

# ========= I/O & TIME =========

OUT      = "/archive/Qian.Xiao/Qian.Xiao/FMS_Wave_Coupling_GOTM_kapp/3D_hurricane/4_deg/Wind_5km_10mi_wave.nc"

DT_SEC   = 600                    # 10-min forcing
N_DAYS   = 3                      # duration
START_TIME_STR = "2011-04-01 00:00:00"
CALENDAR = "gregorian"

# ========= PHYSICS (IDL_HURR_* equivalents) =========
rho_a = 1.2
p_n   = 1.012e5
p_c   = 9.68e4
Rmax  = 5.0e4                 # m
rad_edge    = 10.0
rad_ambient = 15.0
Umax  = 50.0                  # m/s
transpeed   = 5.0             # m/s
transdir    = np.deg2rad(180.0)
X0 = 3.432e6                  # m
Y0 = 9.0e5                    # m
f  = 5.5659e-5                # s^-1
t0_sec = 0

# Inflow angle (Zhang & Uhlhorn 2012)
A0_0 = -14.33; A0_Rnorm = -0.9;  A0_speed = -0.09
A1_0 =  0.14;  A1_Rnorm =  0.04; A1_speed =  0.05
P1_0 = 85.31;  P1_Rnorm =  6.88; P1_speed = -9.60
Deg2Rad = np.pi/180.0

def Cd_from_du10(du10):
    # Sullivan-style piecewise Cd(du10)
    return np.where(du10 < 11.0, 1.2e-3,
           np.where(du10 <= 20.0, (0.49 + 0.065*du10)*1e-3, 1.8e-3))

# ========= GRID (Cartesian domain + angular labels) =========
# physical size (m)
Lx_m = 3.0e6      # 3000 km
Ly_m = 1.8e6      # 1800 km

# model tracer resolution: 5 km → 5000 m
DX_tracer = 5.0e3
DY_tracer = 5.0e3

Nx = int(Lx_m / DX_tracer)   # 600
Ny = int(Ly_m / DY_tracer)   # 360

# 1-D physical coordinates (m) – cell centers
x_m_1d = (np.arange(Nx) + 0.5) * DX_tracer
y_m_1d = (np.arange(Ny) + 0.5) * DY_tracer

# 2-D physical plane (m) for stress_slice
X_m, Y_m = np.meshgrid(x_m_1d, y_m_1d, indexing="xy")

# simple_cartesian_grid angular coordinates (degrees)
# lon_min, lon_max = -13.5, 13.5   # 27° span
# lat_min, lat_max = -8.1, 8.1     # 16.2° span

# dlam = (lon_max - lon_min) / Nx
# dphi = (lat_max - lat_min) / Ny

# Lon_1d_deg = lon_min + (np.arange(Nx) + 0.5) * dlam
# Lat_1d_deg = lat_min + (np.arange(Ny) + 0.5) * dphi
from netCDF4 import Dataset
import numpy as np

g = Dataset("/archive/Qian.Xiao/Qian.Xiao/FMS_Wave_Coupling_GOTM_kapp/3D_hurricane/4_deg/wave_hgrid.nc")
# supergrid -> T点 (600x360)
xT = g.variables["x"][1::2, 1::2]
yT = g.variables["y"][1::2, 1::2]
g.close()

# 1D 坐标（规则网格时行/列恒定）
Lon_1d_deg = xT[0, :].astype(np.float64)   # (600,)
Lat_1d_deg = yT[:, 0].astype(np.float64)   # (360,)

# ---------- TIME (add +DT_SEC tail to avoid bracketing error) ----------
t_sec = np.arange(0, N_DAYS*86400 + DT_SEC, DT_SEC, dtype="int64")  # includes tail
t_min = (t_sec // 60).astype("int32")
Nt    = t_sec.size

# ========= PRECOMPUTE CONSTANTS =========
dp   = p_n - p_c
U_TS = 0.5*transpeed*np.cos(transdir)
V_TS = 0.5*transpeed*np.sin(transdir)
# Holland B (dimensionally consistent form)
B    = (Umax**2) * rho_a * np.e / dp

# U10 at edge for taper (scalar)
rrB_e  = rad_edge**(-B)
tmpA_e = (rrB_e*B) * dp
r_e    = rad_edge * Rmax
tmpB_e = (0.5*r_e*f) * rho_a
num_e  = tmpA_e * np.exp(-rrB_e)
den_e  = tmpB_e + np.sqrt((tmpA_e*rho_a)*np.exp(-rrB_e) + tmpB_e**2)
U10_edge = float(num_e / max(den_e, 1e-20))

# ========= ONE TIME-SLICE SOLVER =========
def stress_slice(tsec):
    XC = X0 + (tsec - t0_sec) * transpeed * np.cos(transdir)
    YC = Y0 + (tsec - t0_sec) * transpeed * np.sin(transdir)

    dX = X_m - XC
    dY = Y_m - YC
    r  = np.hypot(dX, dY)
    rr = r / Rmax

    # U10 magnitude
    U10 = np.zeros_like(r, dtype=np.float64)
    core = (rr > 1e-3) & (rr <= rad_edge)
    if np.any(core):
        rrB = np.power(rr[core], -B)
        tmpA = (rrB*B) * dp
        tmpB = (0.5*r[core]*f) * rho_a
        num  = tmpA * np.exp(-rrB)
        den  = tmpB + np.sqrt((tmpA*rho_a)*np.exp(-rrB) + tmpB**2)
        U10[core] = num / np.maximum(den, 1e-20)

    shell = (rr > rad_edge) & (rr < rad_ambient)
    if np.any(shell):
        fac = (rad_ambient - rr[shell])/(rad_ambient - rad_edge)
        U10[shell] = fac * U10_edge

    # Inflow angle
    Adir = np.arctan2(dY, dX)
    RSTR = np.minimum(rad_edge, rr)
    A0 = (A0_Rnorm*RSTR + A0_speed*Umax) + A0_0
    A1 = -A0*((A1_Rnorm*RSTR + A1_speed*transpeed) + A1_0)
    P1 = ((P1_Rnorm*RSTR + P1_speed*transpeed) + P1_0) * Deg2Rad
    Alph = (A0 - A1*np.cos((transdir - Adir) - P1)) * Deg2Rad
    if np.any(rr > rad_edge):
        taper = np.clip((rad_ambient - rr)/(rad_ambient - rad_edge), 0.0, 1.0)
        Alph = np.where(rr > rad_edge, Alph*taper, Alph)

    # 10 m winds (+0.5 * translation)
    u10x = U10*np.sin(Adir - np.pi - Alph) + U_TS
    u10y = U10*np.cos(Adir - Alph)        + V_TS
    du10 = np.hypot(u10x, u10y)

    # Stress (Pa)
    Cd   = Cd_from_du10(du10)
    Taux = (rho_a * Cd * du10 * u10x).astype(np.float32)
    Tauy = (rho_a * Cd * du10 * u10y).astype(np.float32)

    # Cast winds to float32 for storage
    u10x = u10x.astype(np.float32)
    u10y = u10y.astype(np.float32)

    return Taux, Tauy, u10x, u10y

# ========= WRITE NETCDF (NETCDF4_CLASSIC) =========
os.makedirs(os.path.dirname(OUT), exist_ok=True)
nc = Dataset(OUT, "w", format="NETCDF4_CLASSIC")

# Globals
nc.title       = "Hurricane wind & wind stress (A-grid east/north), 10-min cadence (+tail)"
nc.institution = "GFDL/NOAA (generated)"
nc.source      = "Idealized hurricane wind; lon/lat in degrees (simple_cartesian_grid)"
nc.history     = "Created by wind_writer_meters_coords.py"
nc.Conventions = "CF-1.7"

# Dims
nc.createDimension("time", None)
nc.createDimension("lat", Ny)
nc.createDimension("lon", Nx)

# Time
vT = nc.createVariable("time", "i4", ("time",))
vT.units = f"minutes since {START_TIME_STR}"
vT.calendar = CALENDAR
vT.long_name = "time"
vT.standard_name = "time"
vT.axis = "T"; vT.cartesian_axis = "T"
vT[:] = t_min  # includes the tail record

# 1-D coordinate variables (degrees)
vLat = nc.createVariable("lat", "f8", ("lat",))
vLon = nc.createVariable("lon", "f8", ("lon",))

vLat.units = "degrees_north"
vLon.units = "degrees_east"
vLat.standard_name = "latitude"
vLon.standard_name = "longitude"
vLat.long_name = "latitude"
vLon.long_name = "longitude"
vLat.axis = "Y"; vLon.axis = "X"
vLat.cartesian_axis = "Y"; vLon.cartesian_axis = "X"

vLat[:] = Lat_1d_deg
vLon[:] = Lon_1d_deg

# Fields
FILL = np.float32(9.96921e36)
csy  = min(256, Ny); csx = min(256, Nx)

vtx = nc.createVariable("Taux", "f4", ("time","lat","lon"),
                        zlib=True, complevel=2,
                        chunksizes=(1, csy, csx),
                        fill_value=FILL)
vty = nc.createVariable("Tauy", "f4", ("time","lat","lon"),
                        zlib=True, complevel=2,
                        chunksizes=(1, csy, csx),
                        fill_value=FILL)
vu10 = nc.createVariable("u10", "f4", ("time","lat","lon"),
                         zlib=True, complevel=2,
                         chunksizes=(1, csy, csx),
                         fill_value=FILL)
vv10 = nc.createVariable("v10", "f4", ("time","lat","lon"),
                         zlib=True, complevel=2,
                         chunksizes=(1, csy, csx),
                         fill_value=FILL)

vtx.units = "Pa"; vty.units = "Pa"
vtx.standard_name = "surface_downward_eastward_stress"
vty.standard_name = "surface_downward_northward_stress"
vtx.long_name = "surface wind stress, eastward"
vty.long_name = "surface wind stress, northward"

vu10.units = "m s-1"; vv10.units = "m s-1"
vu10.standard_name = "eastward_wind"
vv10.standard_name = "northward_wind"
vu10.long_name = "10-m eastward wind"
vv10.long_name = "10-m northward wind"

# point to coord vars
for v in (vtx, vty, vu10, vv10):
    v.coordinates = "lat lon"
    v.missing_value = FILL

# Stream out
for k, tsec in enumerate(t_sec):
    tx2d, ty2d, u10x, u10y = stress_slice(int(tsec))
    vtx[k, :, :] = tx2d
    vty[k, :, :] = ty2d
    vu10[k, :, :] = u10x
    vv10[k, :, :] = u10y

nc.close()

# cadence hint for data_table
records_per_day = int(round(86400/DT_SEC))
print(f"[OK] wrote {OUT}")
print(f"[INFO] Nt={Nt} (includes +1 tail); use cadence = -{records_per_day} in data_table.")





In [ ]:

#3DVS1D cases set up: 3D wind files should be modified to have lon and lat instead of Lon and Lat dimension
#then run the 3D wave fields. conver the WW3 output file to MOM6 file that it can read: now, the 3D cases wind and wave files are prepared.
#run the hybrid ST+LT res case again for one-way coupling
#using the theory wind model to generate 360 different wind files and MOM6 wave files to generate 360 wave files. Run 1D hybrid ST, hybrid ST+LT res

#!/usr/bin/env python3
# Hurricane wind + wind-stress for MOM6 data_table ("bilinear")  2m/s input file generate
# - Output: Taux/Tauy/u10/v10(Time, Lat, Lon) with 1-D Lat/Lon in degrees
# - Time axis includes a +1 record tail to avoid "after range" errors

##wind-input files for wave field for 2mps, change the variable names to lon and lat
import os
import numpy as np
from netCDF4 import Dataset

# ========= I/O & TIME =========

OUT      = "/archive/Qian.Xiao/Qian.Xiao/MOM_GLS_LT/3D_hurricane/Wind_5km_10mi_2mps.nc"

DT_SEC   = 600                    # 10-min forcing
N_DAYS   = 3                      # duration
START_TIME_STR = "2011-04-01 00:00:00"
CALENDAR = "gregorian"

# ========= PHYSICS (IDL_HURR_* equivalents) =========
rho_a = 1.2
p_n   = 1.012e5
p_c   = 9.68e4
Rmax  = 5.0e4                 # m
rad_edge    = 10.0
rad_ambient = 15.0
Umax  = 50.0                  # m/s
transpeed   = 2.0             # m/s
transdir    = np.deg2rad(180.0)
X0 = 3.1728e6                  # m
Y0 = 9.0e5                    # m
f  = 5.5659e-5                # s^-1
t0_sec = 0

# Inflow angle (Zhang & Uhlhorn 2012)
A0_0 = -14.33; A0_Rnorm = -0.9;  A0_speed = -0.09
A1_0 =  0.14;  A1_Rnorm =  0.04; A1_speed =  0.05
P1_0 = 85.31;  P1_Rnorm =  6.88; P1_speed = -9.60
Deg2Rad = np.pi/180.0

def Cd_from_du10(du10):
    # Sullivan-style piecewise Cd(du10)
    return np.where(du10 < 11.0, 1.2e-3,
           np.where(du10 <= 20.0, (0.49 + 0.065*du10)*1e-3, 1.8e-3))

# ========= GRID (Cartesian domain + angular labels) =========
# physical size (m)
Lx_m = 3.0e6      # 3000 km
Ly_m = 1.8e6      # 1800 km

# model tracer resolution: 5 km → 5000 m
DX_tracer = 5.0e3
DY_tracer = 5.0e3

Nx = int(Lx_m / DX_tracer)   # 600
Ny = int(Ly_m / DY_tracer)   # 360

# 1-D physical coordinates (m) – cell centers
x_m_1d = (np.arange(Nx) + 0.5) * DX_tracer
y_m_1d = (np.arange(Ny) + 0.5) * DY_tracer

# 2-D physical plane (m) for stress_slice
X_m, Y_m = np.meshgrid(x_m_1d, y_m_1d, indexing="xy")

# simple_cartesian_grid angular coordinates (degrees)
lon_min, lon_max = -13.5, 13.5   # 27° span
lat_min, lat_max = -8.1, 8.1     # 16.2° span

dlam = (lon_max - lon_min) / Nx
dphi = (lat_max - lat_min) / Ny

Lon_1d_deg = lon_min + (np.arange(Nx) + 0.5) * dlam
Lat_1d_deg = lat_min + (np.arange(Ny) + 0.5) * dphi

# ---------- TIME (add +DT_SEC tail to avoid bracketing error) ----------
t_sec = np.arange(0, N_DAYS*86400 + DT_SEC, DT_SEC, dtype="int64")  # includes tail
t_min = (t_sec // 60).astype("int32")
Nt    = t_sec.size

# ========= PRECOMPUTE CONSTANTS =========
dp   = p_n - p_c
U_TS = 0.5*transpeed*np.cos(transdir)
V_TS = 0.5*transpeed*np.sin(transdir)
# Holland B (dimensionally consistent form)
B    = (Umax**2) * rho_a * np.e / dp

# U10 at edge for taper (scalar)
rrB_e  = rad_edge**(-B)
tmpA_e = (rrB_e*B) * dp
r_e    = rad_edge * Rmax
tmpB_e = (0.5*r_e*f) * rho_a
num_e  = tmpA_e * np.exp(-rrB_e)
den_e  = tmpB_e + np.sqrt((tmpA_e*rho_a)*np.exp(-rrB_e) + tmpB_e**2)
U10_edge = float(num_e / max(den_e, 1e-20))

# ========= ONE TIME-SLICE SOLVER =========
def stress_slice(tsec):
    XC = X0 + (tsec - t0_sec) * transpeed * np.cos(transdir)
    YC = Y0 + (tsec - t0_sec) * transpeed * np.sin(transdir)

    dX = X_m - XC
    dY = Y_m - YC
    r  = np.hypot(dX, dY)
    rr = r / Rmax

    # U10 magnitude
    U10 = np.zeros_like(r, dtype=np.float64)
    core = (rr > 1e-3) & (rr <= rad_edge)
    if np.any(core):
        rrB = np.power(rr[core], -B)
        tmpA = (rrB*B) * dp
        tmpB = (0.5*r[core]*f) * rho_a
        num  = tmpA * np.exp(-rrB)
        den  = tmpB + np.sqrt((tmpA*rho_a)*np.exp(-rrB) + tmpB**2)
        U10[core] = num / np.maximum(den, 1e-20)

    shell = (rr > rad_edge) & (rr < rad_ambient)
    if np.any(shell):
        fac = (rad_ambient - rr[shell])/(rad_ambient - rad_edge)
        U10[shell] = fac * U10_edge

    # Inflow angle
    Adir = np.arctan2(dY, dX)
    RSTR = np.minimum(rad_edge, rr)
    A0 = (A0_Rnorm*RSTR + A0_speed*Umax) + A0_0
    A1 = -A0*((A1_Rnorm*RSTR + A1_speed*transpeed) + A1_0)
    P1 = ((P1_Rnorm*RSTR + P1_speed*transpeed) + P1_0) * Deg2Rad
    Alph = (A0 - A1*np.cos((transdir - Adir) - P1)) * Deg2Rad
    if np.any(rr > rad_edge):
        taper = np.clip((rad_ambient - rr)/(rad_ambient - rad_edge), 0.0, 1.0)
        Alph = np.where(rr > rad_edge, Alph*taper, Alph)

    # 10 m winds (+0.5 * translation)
    u10x = U10*np.sin(Adir - np.pi - Alph) + U_TS
    u10y = U10*np.cos(Adir - Alph)        + V_TS
    du10 = np.hypot(u10x, u10y)

    # Stress (Pa)
    Cd   = Cd_from_du10(du10)
    Taux = (rho_a * Cd * du10 * u10x).astype(np.float32)
    Tauy = (rho_a * Cd * du10 * u10y).astype(np.float32)

    # Cast winds to float32 for storage
    u10x = u10x.astype(np.float32)
    u10y = u10y.astype(np.float32)

    return Taux, Tauy, u10x, u10y

# ========= WRITE NETCDF (NETCDF4_CLASSIC) =========
os.makedirs(os.path.dirname(OUT), exist_ok=True)
nc = Dataset(OUT, "w", format="NETCDF4_CLASSIC")

# Globals
nc.title       = "Hurricane wind & wind stress (A-grid east/north), 10-min cadence (+tail)"
nc.institution = "GFDL/NOAA (generated)"
nc.source      = "Idealized hurricane wind; lon/lat in degrees (simple_cartesian_grid)"
nc.history     = "Created by wind_writer_meters_coords.py"
nc.Conventions = "CF-1.7"

# Dims
nc.createDimension("time", None)
nc.createDimension("lat", Ny)
nc.createDimension("lon", Nx)

# Time
vT = nc.createVariable("time", "i4", ("time",))
vT.units = f"minutes since {START_TIME_STR}"
vT.calendar = CALENDAR
vT.long_name = "time"
vT.standard_name = "time"
vT.axis = "T"; vT.cartesian_axis = "T"
vT[:] = t_min  # includes the tail record

# 1-D coordinate variables (degrees)
vLat = nc.createVariable("lat", "f8", ("lat",))
vLon = nc.createVariable("lon", "f8", ("lon",))

vLat.units = "degrees_north"
vLon.units = "degrees_east"
vLat.standard_name = "latitude"
vLon.standard_name = "longitude"
vLat.long_name = "latitude"
vLon.long_name = "longitude"
vLat.axis = "Y"; vLon.axis = "X"
vLat.cartesian_axis = "Y"; vLon.cartesian_axis = "X"

vLat[:] = Lat_1d_deg
vLon[:] = Lon_1d_deg

# Fields
FILL = np.float32(9.96921e36)
csy  = min(256, Ny); csx = min(256, Nx)

vtx = nc.createVariable("Taux", "f4", ("time","lat","lon"),
                        zlib=True, complevel=2,
                        chunksizes=(1, csy, csx),
                        fill_value=FILL)
vty = nc.createVariable("Tauy", "f4", ("time","lat","lon"),
                        zlib=True, complevel=2,
                        chunksizes=(1, csy, csx),
                        fill_value=FILL)
vu10 = nc.createVariable("u10", "f4", ("time","lat","lon"),
                         zlib=True, complevel=2,
                         chunksizes=(1, csy, csx),
                         fill_value=FILL)
vv10 = nc.createVariable("v10", "f4", ("time","lat","lon"),
                         zlib=True, complevel=2,
                         chunksizes=(1, csy, csx),
                         fill_value=FILL)

vtx.units = "Pa"; vty.units = "Pa"
vtx.standard_name = "surface_downward_eastward_stress"
vty.standard_name = "surface_downward_northward_stress"
vtx.long_name = "surface wind stress, eastward"
vty.long_name = "surface wind stress, northward"

vu10.units = "m s-1"; vv10.units = "m s-1"
vu10.standard_name = "eastward_wind"
vv10.standard_name = "northward_wind"
vu10.long_name = "10-m eastward wind"
vv10.long_name = "10-m northward wind"

# point to coord vars
for v in (vtx, vty, vu10, vv10):
    v.coordinates = "lat lon"
    v.missing_value = FILL

# Stream out
for k, tsec in enumerate(t_sec):
    tx2d, ty2d, u10x, u10y = stress_slice(int(tsec))
    vtx[k, :, :] = tx2d
    vty[k, :, :] = ty2d
    vu10[k, :, :] = u10x
    vv10[k, :, :] = u10y

nc.close()

# cadence hint for data_table
records_per_day = int(round(86400/DT_SEC))
print(f"[OK] wrote {OUT}")
print(f"[INFO] Nt={Nt} (includes +1 tail); use cadence = -{records_per_day} in data_table.")

